
# Summing elements of a numpy array


This benchmark compares accuracy and efficiency of several summation algorithms in floating point arithmetics



In [ ]:
def builtin_sum(x):
    return sum(x)

def naive_sum(x):
    s = 0.
    for i in range(x.shape[0]):
        s += x[i]
    return s    
    
nb_naive_sum = nb.jit(naive_sum, **numba_opt_dict)
nb_naive_sum.__name__ = "nb_naive_sum"

def np_sum(x):
    return np.sum(x)

def m_fsum(x):
    return m.fsum(x)

@nb.jit(**numba_opt_dict)
def taylor_poly(n, alpha):
    
    x = np.zeros((n),dtype=np.float64)
    cur_term = 1.
    x[0] = cur_term

    for i in range(1, x.shape[0]):

        cur_term = cur_term * (alpha / i)
        x[i] = cur_term

    return x

def setup(alpha):
    return taylor_poly(int(1e5), -alpha)

@functools.cache
def exact_sum(alpha):
    y = setup(alpha)
    return m.fsum(y)
    
def compute_error(f, x):
    
    ex_res =  exact_sum(-x[1])
    res = f(x)
    rel_err = abs(ex_res - res) / abs(ex_res)
    
    return rel_err + 1e-40


all_alphas = np.array([float(alpha) for alpha in range(500)])

all_funs = [
    naive_sum,
    nb_naive_sum,
    builtin_sum,
    np_sum,
    m_fsum,
]

all_error_funs = { f.__name__ :  functools.partial(compute_error, f) for f in all_funs if f is not m_fsum}

pyquickbench.run_benchmark(
    all_alphas                      ,
    all_error_funs                  ,
    setup = setup                   ,
    mode = "scalar_output"          ,
    filename = error_filename       ,
    ForceBenchmark = ForceBenchmark ,
    title = "Relative error for increasing conditionning"   ,
    show = True                             ,
)

In [ ]:
def prepare_x(n):
    x = np.random.random(n)
    return {'x': x}